# Bokeh plots

This notebook contains advanced visualizations from Bokeh

### Imports

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

### Bokeh Imports

In [8]:
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment

output_notebook()

Loading BokehJS ...

### Load data

In [9]:
census = pd.read_csv("../../data/census/census_county.csv")

### Example plot

In [67]:
#### West
pacific = ["ca", "or", "wa"] # hi and ak belong here too, but bokeh errors with them
mountain = ["mt", "id", "wy", "nv", "ut", "co", "az", "nm"]
west = pacific + mountain

#### South
ws_central = ["tx", "ok", "ar", "la"]
es_central = ["ky", "tn", "ms", "al"]
s_atlantic = ["wv", "md", "dc", "de", "va", "nc", "sc", "ga", "fl"]
south = ws_central + es_central + s_atlantic

#### Midwest
wn_central = ["nd", "sd", "mn", "ne", "ia", "ks", "mo"]
en_central = ["wi", "mi", "il", "in", "oh"]
midwest = wn_central + en_central

#### Northeast
mid_atlantic = ["ny", "pa", "nj"]
new_england = ["me", "vt", "nh", "ma", "ct", "ri"]
northeast = mid_atlantic + new_england

In [71]:
states = northeast

palette.reverse()

cs = {
    code: county for code, county in counties.items() if county["state"] in states
}

county_xs = [county["lons"] for county in cs.values()]
county_ys = [county["lats"] for county in cs.values()]

county_names = [county['name'] for county in cs.values()]
county_rates = [unemployment[county_id] for county_id in cs]
color_mapper = LogColorMapper(palette=palette)

source = ColumnDataSource(data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="Unemployment, 2009", tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Unemployment rate)", "@rate%"),
    ("(Long, Lat)", "($x, $y)"),
]

show(p)